In [1]:
import numpy as np
import os
import tensorflow as tf

np.random.seed(42)

In [ ]:
train_dir = os.path.join('train')
test_dir = os.path.join('test')
val_dir = os.path.join('val')
comb_dir = os.path.join('combined')

In [ ]:
from keras.utils import image_dataset_from_directory
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(48, 48),
    label_mode="categorical",
    color_mode='grayscale',
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    val_dir,
    image_size=(48, 48),
    label_mode="categorical",
    color_mode='grayscale',
    batch_size=32)
test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(48, 48),
    label_mode="categorical",
    color_mode='grayscale',
    batch_size=32)
combined_dataset = image_dataset_from_directory(
    comb_dir,
    image_size=(48, 48),
    label_mode="categorical",
    color_mode='grayscale',
    batch_size=32) 

In [ ]:
from keras import layers
from keras import models

def build_model():

    # we build a Y-network model with two inputs and one output

    input_shape=(48, 48, 1)

    # left branch
    left_inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), padding="same", activation='relu')(left_inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32, (3, 3), padding="same", activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)

    # fill in code below to finsh the left branch
    x = layers.Conv2D(64, (3, 3), padding="same", activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), padding="same", activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)


    # right branch
    right_inputs = layers.Input(shape=input_shape)
    y = layers.Conv2D(32, (3, 3), padding="same", activation='relu')(right_inputs)
    y = layers.BatchNormalization()(y)
    y = layers.Conv2D(32, (3, 3), padding="same", activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.MaxPooling2D((2, 2))(y)
    y = layers.Dropout(0.25)(y)

    # fill in code below to finish the right branch
    y = layers.Conv2D(64, (3, 3), padding="same", activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Conv2D(64, (3, 3), padding="same", activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.MaxPooling2D((2, 2))(y)
    y = layers.Dropout(0.25)(y)


    # concatenate the left and the right branches
    x = layers.concatenate([x, y])
    x = layers.Flatten()(x)

    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # fill in code below
    x = layers.Dropout(0.5)(x)

    # outputs
    outputs = layers.Dense(7, activation='softmax')(x)
    model = models.Model([left_inputs, right_inputs], outputs=outputs)
    return model
model = build_model()
model.summary()

In [ ]:
model.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_dataset, epochs=40, batch_size=32, validation_data=validation_dataset)

In [ ]:
history.history.keys()

Make a directory of the combined data to train the model on

In [ ]:
model = build_model()
model.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(combined_dataset, epochs=40, batch_size=32)

In [ ]:
acc = model.evaluate(test_dataset)
acc